In [30]:
from os import environ

from sqlalchemy import Connection, create_engine, sql
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv

load_dotenv()

True

In [31]:
def get_database_connection() -> Connection:
    """
    Establishes a database connection to the database specified.
    """

    try:
        engine = create_engine(
            f"mssql+pymssql://{environ['DB_USER']}:{environ['DB_PASSWORD']}@{environ['DB_HOST']}/?charset=utf8")

        return engine.connect()

    except ConnectionError as error:
        print(error)


conn = get_database_connection()

In [32]:
def load_all_plant_data(conn: Connection) -> DataFrame:

    conn.execute(sql.text("USE plants;"))

    query = sql.text(
        """SELECT plant_condition.plant_condition_id,
        plant_condition.at, plant_condition.soil_moisture,
        plant_condition.temperature, plant_condition.last_watered,
        plant.plant_id, plant.plant_name,
        plant.scientific_name, botanist.botanist_id, botanist.first_name,
        botanist.surname, botanist.email, botanist.phone_number,
        origin.origin_id, origin.latitude, origin.longitude, origin.region
        FROM s_epsilon.plant_condition
        JOIN s_epsilon.plant ON s_epsilon.plant.plant_id = s_epsilon.plant_condition.plant_id
        JOIN s_epsilon.botanist ON s_epsilon.botanist.botanist_id = s_epsilon.plant.botanist_id
        JOIN s_epsilon.origin ON s_epsilon.origin.origin_id = s_epsilon.plant.origin_id;""")

    conn.execute(sql.text("COMMIT;"))
    res = conn.execute(query).fetchall()
    df = pd.DataFrame(res)

    return df

df = load_all_plant_data(conn)

df.head(5)

,plant_condition_id,at,soil_moisture,temperature,last_watered,plant_id,plant_name,scientific_name,botanist_id,first_name,surname,email,phone_number,origin_id,latitude,longitude,region,continent_id,continent_name
0,2,2023-12-19 11:00:45,25.222584,12.013141,2023-12-18 13:54:32,1,Venus flytrap,N/A,1,Gertrude,Jekyll,gertrude.jekyll@lnhm.co.uk,001-481-273-3691x127,2,33.95015,-118.03917,South Whittier,2,Africa
1,3,2023-12-19 11:00:46,31.762604,9.081111,2023-12-18 14:10:54,2,Corpse flower,N/A,2,Carl,Linnaeus,carl.linnaeus@lnhm.co.uk,(146)994-1635x35992,3,7.65649,4.92235,Efon-Alaaye,3,Asia
2,4,2023-12-19 11:00:48,30.228126,9.959602,2023-12-18 14:50:16,3,Rafflesia arnoldii,N/A,3,Eliza,Andrews,eliza.andrews@lnhm.co.uk,(846)669-6651x75948,1,-19.32556,-41.25528,Resplendor,1,America
3,5,2023-12-19 11:00:49,22.201815,11.305301,2023-12-18 13:16:25,4,Black bat flower,N/A,2,Carl,Linnaeus,carl.linnaeus@lnhm.co.uk,(146)994-1635x35992,4,13.70167,-89.10944,Ilopango,4,Europe
4,10,2023-12-19 11:00:53,27.893563,9.432708,2023-12-18 13:37:24,11,Asclepias Curassavica,Asclepias curassavica,1,Gertrude,Jekyll,gertrude.jekyll@lnhm.co.uk,001-481-273-3691x127,10,20.88953,-156.47432,Kanului,2,Africa


## Average soil moisture

In [33]:
average_soil_moisture = df["soil_moisture"].mean()

average_soil_moisture

27.471041277922733